# 【使い方】

ランタイム>すべてのセルを実行（**Ctrl+F9**）によりすべてのセルを実行し、セル[2]の最後に生成された**URL（Running on public URL）をクリック**して開いてください。（GUIが新しいタブで開かれる）

※セル[1]は実行完了までに約5分、セル[2]は5秒程度を要します。

※このcolabの画面（タブ）は閉じないでください。

※エラー発生などは https://github.com/SatoruMuro/SAM2GUIfor3Drecon/issues にお知らせいただけると助かります。




# [How to use]

Run all cells by selecting Runtime > Run all cells (**Ctrl+F9**), and **click the URL generated at the end of Cell [2] (Running on public URL)** to open it. (The GUI will open in a new tab)

Note: Cell [1] takes about 5 minutes to complete, and Cell [2] takes around 5 seconds.

Note: Please do not close this Colab screen (tab).

If you encounter any errors or issues, please let us know at https://github.com/SatoruMuro/SAM2GUIfor3Drecon/issues. We’d greatly appreciate your feedback!

In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
import torch

print("PyTorch version:", torch.__version__)  # PyTorchのバージョン
print("CUDA available:", torch.cuda.is_available())  # CUDAが利用可能か
print("CUDA version:", torch.version.cuda)  # CUDAのバージョン
print("cuDNN version:", torch.backends.cudnn.version())  # cuDNNのバージョン
print("cuDNN Enabled:", torch.backends.cudnn.enabled)  # cuDNNが有効か


PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
cuDNN version: 90100
cuDNN Enabled: True


In [3]:
import torch

print("Available GPUs:", torch.cuda.device_count())  # GPUの数
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))  # GPUの名前
    print("GPU Memory Allocated:", torch.cuda.memory_allocated(0) / 1024**2, "MB")  # 使用中メモリ
    print("GPU Memory Cached:", torch.cuda.memory_reserved(0) / 1024**2, "MB")  # 予約済みメモリ


Available GPUs: 1
GPU Name: Tesla T4
GPU Memory Allocated: 0.0 MB
GPU Memory Cached: 0.0 MB


In [4]:
using_colab = True

if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())

    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install --no-build-isolation 'git+https://github.com/facebookresearch/sam2.git@2b90b9f5ceec907a1c18123530e92e794ad901a4'


    !mkdir -p ../checkpoints/
    !wget -P ../checkpoints/ https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt

    !mkdir -p ../configs/
    !wget -P ../configs/ https://raw.githubusercontent.com/facebookresearch/sam2/main/sam2/configs/sam2.1/sam2.1_hiera_l.yaml


import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    torch.autocast("cuda", dtype=torch.float16).__enter__()
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nMPSデバイスのサポートは初期段階です。SAM 2はCUDA向けにトレーニングされており、MPS上での数値結果や性能が異なる場合があります。"
    )

from sam2.build_sam import build_sam2_video_predictor
sam2_checkpoint = "../checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)

!pip install transformers huggingface_hub gradio
!pip install svgwrite numpy opencv-python

PyTorch version: 2.6.0+cu124
Torchvision version: 0.21.0+cu124
CUDA is available: True
  Cloning https://github.com/facebookresearch/sam2.git (to revision 2b90b9f5ceec907a1c18123530e92e794ad901a4) to /tmp/pip-req-build-5bvb08c0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/sam2.git /tmp/pip-req-build-5bvb08c0
  Running command git rev-parse -q --verify 'sha^2b90b9f5ceec907a1c18123530e92e794ad901a4'
  Running command git fetch -q https://github.com/facebookresearch/sam2.git 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Resolved https://github.com/facebookresearch/sam2.git to commit 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.8 MB/s eta 0:00:00
  Created wheel for SAM-2: filename=SAM_2-1.0-cp311-cp311-linux_x86_64.whl size=

In [ ]:
!nvcc --version
!nvidia-smi
import gradio as gr
from PIL import Image, ImageDraw
import numpy as np
import cv2
import os
import tempfile
import zipfile
from io import BytesIO
import shutil
import matplotlib.pyplot as plt
import torch

# スクリプト冒頭で starting_index を初期化
starting_index = 0

# アップロードされたJPEG画像ファイル名と画像データを保存するリスト
uploaded_images = []

# 色ラベル一覧 (RGB版)
color_labels = [
    (255, 0, 0), (0, 0, 255), (0, 255, 0), (255, 255, 0),
    (128, 0, 128), (255, 165, 0), (0, 255, 255), (173, 255, 47),
    (128, 128, 128), (0, 128, 128), (255, 192, 203), (255, 20, 147),
    (0, 128, 0), (128, 0, 0), (0, 255, 230), (255, 215, 0),
    (255, 69, 0), (0, 0, 128), (220, 20, 60), (128, 128, 0)
]

# 座標とラベルを保持するリスト
stored_points = []
# 複数のオブジェクトのセグメンテーションマスクを保持するリスト
stored_masks = []
# 現在のオブジェクトのマスクを保持するリスト
current_mask = None
# セグメントしたオブジェクトのリスト
object_list = []

def convert_image_to_jpeg_if_needed(image):
    """画像がJPEGでない場合、JPEGに変換して保存する"""
    try:
        # 既にJPEG形式であれば変換しない
        if isinstance(image, Image.Image):
            if image.format == 'JPEG':
                print(f"Image is already in JPEG format.: {image.filename}")
                return image.filename  # JPEG形式ならファイルパスを返す

            # JPEGに変換
            with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp_file:
                jpeg_image_path = tmp_file.name
                image.convert("RGB").save(jpeg_image_path, "JPEG")
                print(f"Image converted to JPEG and saved as: {jpeg_image_path}")
                return jpeg_image_path
        elif isinstance(image, np.ndarray):
            # NumPy配列をJPEGとして保存
            with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp_file:
                jpeg_image_path = tmp_file.name
                cv2.imwrite(jpeg_image_path, image)
                print(f"NumPy array converted to JPEG and saved as: {jpeg_image_path}")
                return jpeg_image_path
        else:
            raise ValueError(f"Unsupported image type: {type(image)}")

    except Exception as e:
        print(f"Error in converting image to JPEG: {str(e)}")
        raise

def upload_images(files):
    global uploaded_images, starting_index  # starting_index をグローバル変数として宣言
    uploaded_images = []

    # ファイル名に基づいてソートし、最小の番号を取得
    files = sorted(files, key=lambda f: int(os.path.splitext(os.path.basename(f.name))[0][-4:]))
    starting_index = int(os.path.splitext(os.path.basename(files[0].name))[0][-4:])  # 最小番号を取得

    # 保存先フォルダの作成
    image_save_dir = '/content/images'
    if not os.path.exists(image_save_dir):
        os.makedirs(image_save_dir)

    images_with_filenames = []  # 画像と元のファイル名のペアリスト

    for idx, file in enumerate(files):
        image = Image.open(file)
        original_filename = os.path.basename(file.name)  # 元のファイル名を取得

        # 画像フォーマットをデバッグ用に出力
        print(f"Format of uploaded image {idx + 1}: {image.format}")

        # JPEGに変換して保存
        jpeg_image_path = convert_image_to_jpeg_if_needed(image)
        saved_image_path = os.path.join(image_save_dir, f'image_{idx + 1}.jpg')
        image.save(saved_image_path, format="JPEG")
        uploaded_images.append(saved_image_path)

        # 元のファイル名を付けた画像のペアを追加
        images_with_filenames.append((saved_image_path, original_filename))

    # アップロードされた画像でmp4動画を作成
    video_output_path = "/content/output_video.mp4"
    create_video_from_images(uploaded_images, video_output_path)

    # アップロードが完了した時点で、最初の画像を表示
    if uploaded_images:
        first_image = uploaded_images[0]
    else:
        first_image = None

    # グローバル変数 uploaded_images のソート済みリストを Gradio ギャラリー用に返す
    return (f"{len(uploaded_images)} images have been uploaded and saved to {image_save_dir}. Video saved to {video_output_path}.",
            gr.update(maximum=len(uploaded_images), value=1),
            first_image,
            images_with_filenames)  # ギャラリーに画像と元のファイル名のペアを返す

def create_video_from_images(image_list, output_video_path, fps=30):
    if len(image_list) == 0:
        raise ValueError("The image list is empty. Please provide at least one image.")

    first_image = cv2.imread(image_list[0])
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 出力フォーマット
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for image_path in image_list:
        img = cv2.imread(image_path)
        video.write(img)

    video.release()

def display_image(image_index):
    global uploaded_images
    if len(uploaded_images) > 0:
        image_index = int(image_index) - 1  # スライダーは1から始まるため、インデックスとして使う際は1を引く
        if image_index < 0 or image_index >= len(uploaded_images):
            return None  # 範囲外の場合は None を返す
        return uploaded_images[image_index]
    return None


def update_color_start_from_dropdown(selected_index_str):
    global color_start_index
    color_start_index = int(selected_index_str)
    return f"Start color index set to {color_start_index} (RGB: {color_labels[color_start_index]})"




# update_result_on_select 関数が表示する画像を「セグメント結果」にも表示させる
def update_result_on_select(image_index):
    global uploaded_images
    if len(uploaded_images) > 0:
        image_index = int(image_index) - 1  # スライダーのインデックスに合わせて調整
        if image_index < 0 or image_index >= len(uploaded_images):
            return None, None  # 範囲外の場合は None を2つ返す

        selected_image = uploaded_images[image_index]

        # 画像を読み込んで BGR から RGB に変換
        image_np = cv2.imread(selected_image)
        if image_np is None:
            raise ValueError(f"Failed to load the image.: {selected_image}")

        # BGR から RGB に変換
        image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

        # 初期値として50%の位置にラインを描画して「セグメント結果」に表示
        initial_image_with_lines = update_image_with_lines(image_np, 50, 50)

        # 選択された画像と、セグメント結果に表示する画像を返す
        return image_np, initial_image_with_lines  # 両方の出力を返す
    return None, None  # 画像がない場合は None を2つ返す

last_used_coords = []  # 座標を複数記録できるようにリストに変更
stored_masks = []  # 複数マスクを保持するリスト

# AddとRemoveの座標を保持するリスト
current_object_points = []  # (x座標, y座標, ラベル) のタプルで保持

top_left = None
bottom_right = None

def set_top_left(x_percent, y_percent):
    global top_left
    top_left = (x_percent, y_percent)
    return f"Top left set at ({x_percent}, {y_percent})"

def set_bottom_right(x_percent, y_percent, image):
    global top_left, bottom_right, predictor, current_mask, stored_masks

    # 画像が None の場合はエラーメッセージを返す
    if image is None:
        print("No image provided.")
        return None, "No image provided. Please upload an image."

    # 画像がJPEG形式であることを確認し、必要なら変換して一時ディレクトリに保存
    jpeg_image_path = convert_image_to_jpeg_if_needed(image)

    # JPEGファイルを NumPy 配列に変換
    image_np = cv2.imread(jpeg_image_path)
    if image_np is None:
        raise ValueError(f"Failed to load the JPEG file: {jpeg_image_path}")

    # 一時ディレクトリに保存されたJPEGファイルを使う
    image_dir = tempfile.mkdtemp()  # 一時ディレクトリ作成
    image_filename = os.path.join(image_dir, "0.jpg")  # フレーム名に「0.jpg」を使用
    cv2.imwrite(image_filename, image_np)  # 画像を保存
    print(f"Image saved to: {image_filename}")

    # 右下の座標を設定
    bottom_right = (x_percent, y_percent)
    print(f"Bottom-right coordinates set: {bottom_right}")

    # 両方の座標が設定されているか確認
    if top_left is not None and bottom_right is not None:
        try:
            # 画像のサイズを取得
            height, width, _ = image_np.shape
            print(f"Image dimensions - Width: {width}, Height: {height}")

            # ボックスの座標をピクセル単位に変換
            box = np.array([
                int(width * (top_left[0] / 100)), int(height * (top_left[1] / 100)),
                int(width * (bottom_right[0] / 100)), int(height * (bottom_right[1] / 100))
            ], dtype=np.float32)
            print(f"Box coordinates: {box}")

            # 推論状態の初期化（JPEGファイルをディレクトリから読み込む）
            inference_state = predictor.init_state(video_path=image_dir)  # ディレクトリを指定
            print("Inference state initialized.")

            try:
                # ボックスプロンプトを用いたセグメンテーション予測
                frame_idx = 0  # フレームインデックスとして整数を指定
                _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
                    inference_state=inference_state,  # 推論状態
                    frame_idx=frame_idx,  # フレームインデックスとして整数を渡す
                    obj_id=1,  # オブジェクトID
                    box=box  # ボックス座標
                )

                # マスクを展開し、1次元目を削除して2次元に変換
                masks = (out_mask_logits[0] > 0.0).cpu().numpy().squeeze()  # squeeze()で1次元を削除
                print(f"Generated mask shape: {masks.shape}")

            except Exception as e:
                print(f"Segmentation prediction failed: {str(e)}")
                return None, f"Segmentation prediction failed: {str(e)}"

            # マスクの確認
            if masks is None or masks.size == 0:
                print("No masks were generated.")
                raise ValueError("The segmentation mask was not generated.")

            # 現在のマスクを保持
            current_mask = masks
            print("Current mask saved.")

            # 既存のマスクが空かどうか確認
            if not stored_masks:
                print("No previous masks stored.")
            else:
                print(f"Stored masks count: {len(stored_masks)}")

            # 既存のマスクと現在のマスクを合わせて描画
            output_image = apply_all_masks2(image_np, stored_masks + [current_mask])
            if output_image is not None:
                print("Mask successfully applied to image.")
            else:
                print("Mask application to image failed.")

            # 画像データとメッセージを個別に返す
            return output_image, "Segmentation succeeded."

        except Exception as e:
            print(f"Error in segmentation process: {str(e)}")
            return None, f"An error occurred: {str(e)}"
    else:
        print(f"Coordinates not set properly. Top-left: {top_left}, Bottom-right: {bottom_right}")
        return None, "Please set both top-left and bottom-right coordinates."

def update_segmentation_result_on_undo(image_np, stored_masks, x_percent, y_percent):
    try:
        # マスクが適用された画像を生成
        output_image = apply_all_masks5(image_np, stored_masks)

        # マスク適用後の画像にラインを描画
        output_image_with_lines = draw_lines_and_points(output_image, x_percent, y_percent, stored_points)

        # 画像を返す
        return output_image_with_lines

    except Exception as e:
        print(f"Error in update_segmentation_result_on_undo: {str(e)}")
        return None

def apply_all_masks5(image, masks):
    try:
        blended_image = np.array(image, dtype=np.uint8)

        for idx, mask in enumerate(masks):
            if mask is None or mask.size == 0:
                print(f"The mask was not applied.: {idx}")
                continue  # マスクがない場合はスキップ

            # 色ラベルから対応するBGRの色を選択 (最大20個まで対応)
            if idx < len(color_labels):
                color_bgr = color_labels[(color_start_index + idx) % len(color_labels)]
            else:
                color_bgr = (255, 255, 255)  # ラベルリスト外の場合は白を適用

            # マスク部分を半透明で色付け
            mask_bgr = np.zeros_like(blended_image, dtype=np.uint8)
            mask_bgr[mask == 1] = color_bgr  # マスク部分にBGRの色を適用

            # 半透明度の設定
            alpha = 0.5  # 半透明度
            blended_image[mask == 1] = cv2.addWeighted(
                blended_image[mask == 1], 1 - alpha, mask_bgr[mask == 1], alpha, 0
            )  # マスクを半透明で重ねる

            # 輪郭を追加
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(blended_image, contours, -1, (255, 255, 255), 2)  # 白い輪郭をBGRで描画

        print(f"The mask was applied.: {len(masks)} masks.")

        return blended_image
    except Exception as e:
        print(f"apply_all_masks5 error.: {str(e)}")
        raise

# オブジェクトカウント用のグローバル変数
object_counter = 0
# 色ラベル開始のグローバル変数
color_start_index = 0  # 色ラベルの開始インデックス

def undo_box_selection(image_np, output_image, x_percent, y_percent):
    global top_left, bottom_right, current_mask, object_counter

    # デバッグメッセージを追加して出力画像の状態を確認
    print(f"Undo called for object {object_counter} with x_percent: {x_percent}, y_percent: {y_percent}")

    # オブジェクトカウンターに基づいて、使用する画像を決定
    if object_counter == 0:
        image_to_use = image_np  # オブジェクト1の場合
    else:
        image_to_use = output_image.copy()  # オブジェクト2以降ではoutput_imageのコピーを使用

    # 画像が None の場合のエラーハンドリング
    if image_to_use is None:
        return gr.update(value=None), gr.update(value="No image available to reset.")

    # 座標のリセット
    top_left = None
    bottom_right = None
    current_mask = None  # current_mask をクリア
    print("Undo action: current_mask is reset to None")

    # 新しいoutput_imageを作成してstored_masksのみ描画
    output_image_with_lines = apply_all_masks5(image_to_use, stored_masks)  # current_maskは含まない

    # スライダーの値を強制的に+1にする（X座標の値を+1）
    new_x_percent = min(100, x_percent + 1)  # 最大値は100に制限
    print(f"X slider updated to: {new_x_percent}")

    # スライダーの値を更新する
    x_slider_update = gr.update(value=new_x_percent)

    # スライダーの変更イベントをシミュレートして、画像を更新
    output_image_with_lines = update_image_with_lines(output_image_with_lines, new_x_percent, y_percent)

    # 再描画を強制し、重ね合わせが残らないようにする
    print("Final output image with masks and lines ready for display.")

    # output_image_with_linesをGradioのインターフェースで表示する
    return output_image_with_lines, gr.update(value="Box selection has been undone."), x_slider_update

def update_image_with_lines(image, x_percent, y_percent):
    global stored_masks, current_mask

    # 画像がファイルパスで渡されている場合、画像を読み込む
    if isinstance(image, str):
        image_np = cv2.imread(image)  # ファイルパスを読み込む
    elif isinstance(image, Image.Image):
        image_np = np.array(image.convert("RGB"))
    else:
        image_np = image  # すでに NumPy 配列ならそのまま使用

    # セグメンテーションマスクを適用　#apply_all_masks2を使ってみる
    output_image = apply_all_masks2(image_np, stored_masks + ([current_mask] if current_mask is not None else []))
    return draw_lines_and_points(output_image, x_percent, y_percent, stored_points)

def draw_lines_and_points(image, current_x, current_y, stored_points):
    if image is None:
        return "Error: No image data"

    # 画像が PIL 形式か NumPy 配列かによってサイズの取得方法を変える
    if isinstance(image, Image.Image):
        width, height = image.size
        processed_image = image.copy()  # PILの場合、コピーして描画
    else:
        height, width, _ = image.shape
        processed_image = Image.fromarray(image)  # NumPyの場合、PILに変換

    draw = ImageDraw.Draw(processed_image)

    # 垂直線と水平線を描画 (現在のスライダー位置)
    x_coord = int(width * (current_x / 100))
    y_coord = int(height * (current_y / 100))
    draw.line([(x_coord, 0), (x_coord, height)], fill="yellow", width=3)
    draw.line([(0, y_coord), (width, y_coord)], fill="yellow", width=3)

    # 記憶された座標に青い点（Add）と赤い点（Remove）を描画
    for (x_percent, y_percent, label) in stored_points:
        x_point = int(width * (x_percent / 100))
        y_point = int(height * (y_percent / 100))
        color = "blue" if label == 1 else "red"  # Addは青、Removeは赤
        draw.ellipse([(x_point - 5, y_point - 5), (x_point + 5, y_point + 5)], fill=color, outline=color)

    # 最後に NumPy 配列に戻す
    return np.array(processed_image)

def apply_all_masks(image, masks):
    try:
        blended_image = np.array(image, dtype=np.uint8)

        for idx, mask in enumerate(masks):
            if mask is None or mask.size == 0:
                print(f"The mask was not applied.: {idx}")
                continue

            # 色ラベルから対応するBGRの色を選択 (最大20個まで対応)
            if idx < len(color_labels):
                # RBGに変換 (BGRの値からRとBを入れ替える)
                color_bgr = color_labels[(color_start_index + idx) % len(color_labels)]
                color_rgb = (color_bgr[2], color_bgr[1], color_bgr[0])  # BとRを入れ替えてRGBに変換
            else:
                color_rgb = (255, 255, 255)  # ラベルリスト外の場合は白を適用

            # マスク部分を半透明で色付け (RGBで色を適用)
            mask_rgb = np.zeros_like(blended_image, dtype=np.uint8)
            mask_rgb[mask == 1] = color_rgb  # マスク部分に色を適用 (RGB)

            # 半透明度の設定
            alpha = 0.5  # 半透明度
            blended_image[mask == 1] = cv2.addWeighted(
                blended_image[mask == 1], 1 - alpha, mask_rgb[mask == 1], alpha, 0
            )  # マスクを半透明で重ねる

            # 輪郭を追加
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(blended_image, contours, -1, (255, 255, 255), 2)  # 白い輪郭を描画

        print(f"The mask was applied.: {len(masks)} masks.")

        return blended_image
    except Exception as e:
        print(f"apply_all_masks error.: {str(e)}")
        raise

def reset_lines_on_add_object(image):
    # 画像が NumPy 配列か PIL 形式か確認して NumPy 配列に変換
    if isinstance(image, Image.Image):
        image_np = np.array(image.convert("RGB"))
    else:
        image_np = image  # すでに NumPy 配列の場合はそのまま使用

    # 画像のサイズを取得
    height, width, _ = image_np.shape

    # 画像のコピーを作成し、中央に線を描画
    processed_image = image_np.copy()
    x_coord = width // 2
    y_coord = height // 2

    # 垂直線と水平線を描画 (黄色の線)
    cv2.line(processed_image, (x_coord, 0), (x_coord, height), (255, 255, 0), 2)  # 垂直線
    cv2.line(processed_image, (0, y_coord), (width, y_coord), (255, 255, 0), 2)  # 水平線

    return processed_image

def apply_all_masks2(image, masks):
    try:
        blended_image = np.array(image, dtype=np.uint8)

        for idx, mask in enumerate(masks):
            if mask is None or mask.size == 0:
                print(f"The mask was not applied.: {idx}")
                continue  # マスクがない場合はスキップ

            # 色ラベルから対応するBGRの色を選択 (最大20個まで対応)
            if idx < len(color_labels):
                color_bgr = color_labels[(color_start_index + idx) % len(color_labels)]
            else:
                color_bgr = (255, 255, 255)  # ラベルリスト外の場合は白を適用

            # マスク部分を半透明で色付け
            mask_bgr = np.zeros_like(blended_image, dtype=np.uint8)
            mask_bgr[mask == 1] = color_bgr  # マスク部分にBGRの色を適用

            # 半透明度の設定
            alpha = 0.5  # 半透明度
            blended_image[mask == 1] = cv2.addWeighted(
                blended_image[mask == 1], 1 - alpha, mask_bgr[mask == 1], alpha, 0
            )  # マスクを半透明で重ねる

            # 輪郭を追加
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(blended_image, contours, -1, (255, 255, 255), 2)  # 白い輪郭をBGRで描画

        print(f"The mask was applied.: {len(masks)} masks.")

        return blended_image
    except Exception as e:
        print(f"apply_all_masks2 error.: {str(e)}")
        raise

def reset_lines_with_slider_positions(image, x_percent, y_percent):
    # 画像が NumPy 配列か PIL 形式か確認して NumPy 配列に変換
    if isinstance(image, Image.Image):
        image_np = np.array(image.convert("RGB"))
    else:
        image_np = image  # すでに NumPy 配列の場合はそのまま使用

    # 画像のサイズを取得
    height, width, _ = image_np.shape

    # スライダーの値に基づいて座標を計算
    x_coord = int(width * (x_percent / 100))
    y_coord = int(height * (y_percent / 100))

    # 画像のコピーを作成し、スライダーの位置に線を描画
    processed_image = image_np.copy()
    cv2.line(processed_image, (x_coord, 0), (x_coord, height), (255, 255, 0), 2)  # 垂直線
    cv2.line(processed_image, (0, y_coord), (width, y_coord), (255, 255, 0), 2)  # 水平線

    return processed_image

stored_boxes = []  # ボックス情報を保存するリスト

def complete_and_add_object(image, x_percent, y_percent):
    global stored_masks, current_mask, stored_points, object_list, object_counter, current_object_points

    if current_mask is not None:
        # 現在のマスクを保存し、オブジェクトカウンターをインクリメント
        stored_masks.append(current_mask)
        object_counter += 1  # カウンターを1ずつ増やす
        object_list.append(f"Object {object_counter}")  # カウンターを使ってオブジェクトを追加

        # ボックス情報を保存（ここでtop_leftとbottom_rightを使う）
        if top_left is not None and bottom_right is not None:
            stored_boxes.append((top_left, bottom_right))  # ボックス情報を保存

        # `current_mask` のリセットを遅らせ、トラッキングまで保持
        stored_points = []  # 座標をリセット
        current_mask = None  # 新しいオブジェクトのためにリセット
        current_object_points = []  # 現在のオブジェクトの座標もリセット

    # 画像が PIL 形式か NumPy 配列かを確認して適切に処理
    if isinstance(image, Image.Image):
        image_np = np.array(image.convert("RGB"))
    else:
        image_np = image  # すでに NumPy 配列ならそのまま使用

    # `apply_all_masks2` を使用して既存のマスクに新しいマスクを重ねて表示
    output_image = apply_all_masks2(image_np, stored_masks)

    # オブジェクトリストの更新
    object_list_text = "\n".join(object_list)

    # 新しいオブジェクトの追加時にスライダーの値に基づいて線を表示
    output_image_with_lines = reset_lines_with_slider_positions(output_image, x_percent, y_percent)

    return output_image_with_lines, "Moved to the segmentation of the new object.", object_list_text

import shutil
import os

def reset_all(image):
    global stored_points, stored_masks, current_mask, object_list, last_used_coords, current_object_points, object_counter, uploaded_images, predictor

    # 既存のリストや変数をリセット
    stored_points = []
    stored_masks = []
    current_mask = None
    object_list = []
    last_used_coords = []
    current_object_points = []
    stored_boxes = []
    object_counter = 0
    uploaded_images = []  # アップロードされた画像リストをリセット

    # フォルダの削除処理をまとめる
    folders_to_delete = [
        '/content/masks',
        '/content/maskcolors',
        '/content/images',
        '/content/segmentation_results',
        '/content/videos',
        '/content/videos_reversed'
    ]
    for folder in folders_to_delete:
        if os.path.exists(folder):
            shutil.rmtree(folder)

    # 動画ファイルの削除
    video_output_path = '/content/output_video.mp4'
    if os.path.exists(video_output_path):
        os.remove(video_output_path)

    # ZIPファイルの削除
    zip_filepath_masks = '/content/segmented_images.zip'
    zip_filepath_maskcolors = '/content/mask_color_images.zip'
    if os.path.exists(zip_filepath_masks):
        os.remove(zip_filepath_masks)
    if os.path.exists(zip_filepath_maskcolors):
        os.remove(zip_filepath_maskcolors)

    # `predictor`の状態リセット
    if 'predictor' in globals():
        predictor.reset_state()

    # テンポラリディレクトリやファイルを削除
    temp_dir = tempfile.gettempdir()
    for temp_file in os.listdir(temp_dir):
        temp_file_path = os.path.join(temp_dir, temp_file)
        try:
            if os.path.isfile(temp_file_path):
                os.unlink(temp_file_path)  # 一時ファイルを削除
            elif os.path.isdir(temp_file_path):
                shutil.rmtree(temp_file_path)  # 一時ディレクトリを削除
        except Exception as e:
            print(f"Failed to delete temporary file or directory {temp_file_path}: {str(e)}")

    # オブジェクトリストをリセットした際にテキスト表示も空にする
    return image, "", "", ""

def apply_segmentation_to_image(image):
    global image_predictor

    # 画像をセグメンテーションする
    if isinstance(image, Image.Image):
        image_np = np.array(image.convert("RGB"))
    else:
        image_np = image

    # セグメンテーション実行
    image_predictor.set_image(image_np)
    masks, scores, logits = image_predictor.predict(point_coords=np.array([[50, 50]]), point_labels=np.array([1]), multimask_output=True)

    # 結果を適用
    output_image = apply_all_masks(image_np, [masks[0]])

    return output_image

# フォルダ作成用の関数
def create_mask_folder():
    mask_folder = '/content/masks'
    if not os.path.exists(mask_folder):
        os.makedirs(mask_folder)
    return mask_folder

# 画像を保存する関数
def save_masked_image_to_disk(image_np, filename):
    img_pil = Image.fromarray(image_np)
    img_pil.save(filename, format="JPEG")

# ZIPファイルに保存されたマスク画像をまとめる関数
def create_zip_from_masks(zip_filename="segmented_images.zip"):
    mask_folder = create_mask_folder()
    zip_filepath = f"/content/{zip_filename}"

    with zipfile.ZipFile(zip_filepath, 'w') as zip_file:
        for root, _, files in os.walk(mask_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, arcname=file)  # ZIPファイル内の相対パスを保持

    return zip_filepath

# フォルダ作成用の関数
def create_mask_folder():
    print("create_mask_folder called")  # デバッグメッセージ
    mask_folder = '/content/masks'
    if not os.path.exists(mask_folder):
        os.makedirs(mask_folder)
        print(f"Folder created at: {mask_folder}")  # フォルダ作成確認
    return mask_folder

# マスクカラー画像用フォルダ作成関数
def create_mask_color_folder():
    print("create_mask_color_folder called")  # デバッグメッセージ
    mask_color_folder = '/content/maskcolors'
    if not os.path.exists(mask_color_folder):
        os.makedirs(mask_color_folder)
        print(f"Folder created at: {mask_color_folder}")  # フォルダ作成確認
    return mask_color_folder

# 画像を保存する関数
def save_masked_image_to_disk(image_np, filename):
    print(f"save_masked_image_to_disk called for {filename}")  # デバッグメッセージ
    img_pil = Image.fromarray(image_np)
    img_pil.save(filename, format="JPEG")
    print(f"Image saved to {filename}")  # 保存確認

def save_mask_color_image_to_disk(image_np, masks, filename):
    print(f"save_mask_color_image_to_disk called for {filename}")  # デバッグメッセージ
    """マスクカラー画像を生成し、黒背景に描画して保存"""
    mask_color_image = np.zeros_like(image_np, dtype=np.uint8)  # 黒背景を作成

    for idx, mask in enumerate(masks):
        if mask is None or mask.size == 0:
            print(f"The mask was not applied.: {idx}")
            continue

        # マスクが2次元か確認し、もし次元が1次元であればリサイズ
        if mask.ndim != 2:
            mask = np.squeeze(mask)  # マスクが2次元になるように次元を縮小

        # 色ラベルから対応する色を選択
        if idx < len(color_labels):
            color = color_labels[(color_start_index + idx) % len(color_labels)]
        else:
            color = (255, 255, 255)  # ラベルリスト外の場合は白を適用

        # マスク部分を完全に塗りつぶす
        mask_color_image[mask == 1] = color  # マスク部分に色を適用 (透過なし)

    # 保存
    img_pil = Image.fromarray(mask_color_image)
    img_pil.save(filename, format="PNG")   # ここをJPEGからPNGに変更
    print(f"Mask color image saved to {filename}")  # 保存確認

# ZIPファイルに保存されたマスク画像をまとめる関数
def create_zip_from_masks(zip_filename="segmented_images.zip"):
    segmentation_results_folder = '/content/segmentation_results'
    zip_filepath = f"/content/{zip_filename}"

    with zipfile.ZipFile(zip_filepath, 'w') as zip_file:
        for root, _, files in os.walk(segmentation_results_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, arcname=file)  # ZIPファイル内の相対パスを保持

    return zip_filepath

# マスクカラーの画像をまとめるZIPファイル生成関数
def create_zip_from_maskcolors(zip_filename="mask_color_images.zip"):
    print("create_zip_from_maskcolors called")  # デバッグメッセージ
    mask_color_folder = create_mask_color_folder()
    zip_filepath = f"/content/{zip_filename}"

    with zipfile.ZipFile(zip_filepath, 'w') as zip_file:
        for root, _, files in os.walk(mask_color_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, arcname=file)  # ZIPファイル内の相対パスを保持

    print(f"ZIP file for mask colors created at {zip_filepath}")  # ZIPファイル作成確認
    return zip_filepath

import svgwrite

# SVGファイルを生成する関数
def create_svg_from_mask(image_np, masks, filename):
    height, width, _ = image_np.shape
    dwg = svgwrite.Drawing(filename, profile='tiny', size=(width, height))

    # 指定されたマスクカラーを使用してベクター化
    for idx, mask in enumerate(masks):
        if mask is None or mask.size == 0:
            continue

        # 対応するカラーの取得（RGBをそのまま使用）
        color_rgb = color_labels[(color_start_index + idx) % len(color_labels)]
        hex_color = '#{:02x}{:02x}{:02x}'.format(color_rgb[0], color_rgb[1], color_rgb[2])

        # 輪郭の検出と追加
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            points = [(int(point[0][0]), int(point[0][1])) for point in contour]
            dwg.add(dwg.polygon(points, fill=hex_color))

    # SVGファイルの保存
    dwg.save()
    print(f"SVG file created: {filename}")

# SVGファイルをまとめるZIPファイル生成関数
def create_zip_from_svgs(zip_filename="mask_svgs.zip"):
    svg_folder = '/content/mask_svgs'
    zip_filepath = f"/content/{zip_filename}"

    with zipfile.ZipFile(zip_filepath, 'w') as zip_file:
        for root, _, files in os.walk(svg_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, arcname=file)  # ZIPファイル内の相対パスを保持

    print(f"ZIP file for SVGs created at {zip_filepath}")
    return zip_filepath

# グレースケールの色ラベルをRGB形式で定義
grayscale_labels = [
    (255, 255, 255), (248, 248, 248), (237, 237, 237), (226, 226, 226),
    (215, 215, 215), (204, 204, 204), (193, 193, 193), (182, 182, 182),
    (171, 171, 171), (160, 160, 160), (149, 149, 149), (138, 138, 138),
    (127, 127, 127), (116, 116, 116), (105, 105, 105), (94, 94, 94),
    (83, 83, 83), (72, 72, 72), (61, 61, 61), (50, 50, 50)
]

# グレースケールマスクを保存する関数（RGBとして処理）
def save_grayscale_mask(image_np, masks, filename):
    print(f"save_grayscale_mask called for {filename}")
    """グレースケールマスク画像を生成し、黒背景に描画して保存"""

    # 黒背景を作成（3次元のRGB画像と同じ形にする）
    grayscale_mask_image = np.zeros_like(image_np, dtype=np.uint8)
    print(f"Initial grayscale mask created with shape: {grayscale_mask_image.shape}")

    for idx, mask in enumerate(masks):
        if mask is None or mask.size == 0:
            print(f"The mask was not applied.: {idx}")
            continue

        # マスクが2次元か確認し、もし次元が1次元であればリサイズ
        if mask.ndim != 2:
            mask = np.squeeze(mask)  # マスクが2次元になるように次元を縮小
        print(f"Processing mask {idx} with shape {mask.shape}")

        # グレースケールラベルから対応する色（RGB形式）を選択
        if idx < len(grayscale_labels):
            grayscale_value = grayscale_labels[(color_start_index + idx) % len(grayscale_labels)]
        else:
            grayscale_value = (10, 10, 10)  # ラベルリスト外の場合は(10, 10, 10)を適用

        # マスク部分にグレースケール値を適用（RGB形式で完全に塗りつぶす）
        grayscale_mask_image[mask == 1] = grayscale_value

    # 保存
    img_pil = Image.fromarray(grayscale_mask_image)
    img_pil.save(filename, format="PNG")
    print(f"Grayscale mask image saved to {filename}")

# グレースケールマスク画像をまとめるZIPファイル生成関数
def create_zip_from_grayscale_masks(zip_filename="grayscale_masks.zip"):
    grayscale_folder = '/content/grayscale_masks'
    zip_filepath = f"/content/{zip_filename}"

    if not os.path.exists(grayscale_folder):
        os.makedirs(grayscale_folder)

    with zipfile.ZipFile(zip_filepath, 'w') as zip_file:
        for root, _, files in os.walk(grayscale_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, arcname=file)  # ZIPファイル内の相対パスを保持

    print(f"ZIP file for grayscale masks created at {zip_filepath}")
    return zip_filepath

def apply_all_masks3(image, masks, frame_idx, svg_folder):
    global starting_index  # starting_indexをグローバル変数として宣言
    try:
        blended_image = np.array(image, dtype=np.uint8)
        image_height, image_width = blended_image.shape[:2]


        # SVGファイルの保存先を設定
        svg_filename = os.path.join(svg_folder, f"mask{starting_index + frame_idx:04}.svg")
        dwg = svgwrite.Drawing(svg_filename, profile='tiny', size=(image_width, image_height))
        dwg.add(dwg.rect(insert=(0, 0), size=(image_width, image_height), fill='black'))  # 背景を黒に設定

        # マスクの適用処理
        mask_applied = False  # マスクが適用されたかを確認するためのフラグ
        for idx, mask in enumerate(masks):
            if mask is None or mask.size == 0:
                print(f"The mask at index {idx} is None or empty. Skipping...")
                continue  # マスクがない場合はスキップ

            # マスクがbool型の場合、uint8に変換
            if mask.dtype == bool:
                mask = mask.astype(np.uint8)

            # デバッグ出力: マスクの状態を確認
            print(f"Original mask at index {idx}: {mask}")

            # マスクのサイズを画像に合わせてリサイズ
            if mask.ndim > 2:
                mask = mask.squeeze()  # 余計な次元を削除

            mask_resized = cv2.resize(mask, (image_width, image_height))

            # デバッグ出力: リサイズ後のマスクを確認
            print(f"Resized mask at index {idx}: {mask_resized}")

            # デバッグ出力: マスクと画像のサイズを確認
            print(f"Mask size: {mask_resized.shape}, Image size: {blended_image.shape}")

            # マスクがすべて0の場合スキップ
            if np.all(mask_resized == 0):
                print(f"Mask at index {idx} contains only zeros. Skipping...")
                continue

            # 色ラベルから対応するBGRの色を選択 (最大20個まで対応)
            if idx < len(color_labels):
                color_bgr = color_labels[(color_start_index + idx) % len(color_labels)]
            else:
                color_bgr = (255, 255, 255)  # ラベルリスト外の場合は白を適用

            # デバッグ出力: 使用する色を確認
            print(f"Color BGR at index {idx}: {color_bgr}")

            hex_color = '#{:02x}{:02x}{:02x}'.format(color_bgr[0], color_bgr[1], color_bgr[2])  # RGBカラーをそのまま使用

            # 2次元のマスクを3次元に拡張する
            mask_bgr = np.zeros_like(blended_image, dtype=np.uint8)
            mask_3d = np.repeat(mask_resized[:, :, np.newaxis], 3, axis=2)  # 2次元マスクを3チャンネルに拡張

            # デバッグ出力: マスクの拡張後を確認
            print(f"Expanded mask at index {idx}: {mask_3d.shape}")

            # マスク部分を色付け
            mask_bgr[mask_resized == 1] = np.array(color_bgr)

            # デバッグ出力: 色付け後のマスクを確認
            print(f"Mask BGR at index {idx}: {mask_bgr}")

            # 半透明度の設定
            alpha = 0.5  # 半透明度
            blended_image[mask_resized == 1] = cv2.addWeighted(
                blended_image[mask_resized == 1], 1 - alpha, mask_bgr[mask_resized == 1], alpha, 0
            )

            # 輪郭を追加
            contours, _ = cv2.findContours(mask_resized.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(blended_image, contours, -1, (255, 255, 255), 2)  # 白い輪郭を描画

            # 輪郭をSVGファイルに追加
            contours, _ = cv2.findContours(mask_resized.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                points = [(int(point[0][0]), int(point[0][1])) for point in contour]
                dwg.add(dwg.polygon(points, fill=hex_color))

            # # SVGファイルを保存
            # dwg.save()
            # print(f"SVG file created for frame {frame_idx}: {svg_filename}")
            # # --- 追加終了 ---

        # マスクが無い場合でも背景のみのSVGファイルを保存
        if mask_applied or not mask_applied:
            dwg.save()
            print(f"SVG file created for frame {frame_idx}: {svg_filename}")

        print(f"The mask was applied.: {len(masks)} masks.")

        return blended_image
    except Exception as e:
        print(f"apply_all_masks3 error: {str(e)} at mask index {idx}")
        raise

# フレームの数を確認する関数
def count_frames(video_dir):
    frame_names = [p for p in os.listdir(video_dir) if os.path.splitext(p)[-1].lower() in [".jpg", ".jpeg"]]
    frame_names.sort(key=lambda p: int(os.path.splitext(p)[0]))
    print(f"Total number of frames: {len(frame_names)}")
    return frame_names

def create_mask_from_box(box, image_shape):
    """ボックスの座標からマスクを作成します"""
    mask = np.zeros(image_shape[:2], dtype=np.float32)
    x_min, y_min, x_max, y_max = map(int, box)
    mask[y_min:y_max, x_min:x_max] = 1.0
    return mask

def process_video_with_propagation(video_path, last_used_coords, stored_boxes, image_index):
    # フォルダの作成
    if not os.path.exists("./videos"):
        os.makedirs("./videos")

    # FFmpegを使って動画をJPEGフレームに変換
    os.system(f"ffmpeg -i {video_path} -q:v 2 -start_number 0 ./videos/%05d.jpg")

    # フレームの数を確認
    video_dir = "./videos"
    frame_names = count_frames(video_dir)

    # セグメンテーション処理を行うための状態を初期化
    inference_state = predictor.init_state(video_path=video_dir)
    predictor.reset_state(inference_state)

    # 指定されたimage_indexに対応するフレームを読み込み
    selected_frame_path = os.path.join(video_dir, frame_names[image_index - 1])
    image_np = np.array(Image.open(selected_frame_path))

    if image_np is not None:
        frame_height, frame_width = image_np.shape[:2]

        for obj_idx, (top_left, bottom_right) in enumerate(stored_boxes):
            obj_id = obj_idx + 1

            box = np.array([
                int(frame_width * (top_left[0] / 100)), int(frame_height * (top_left[1] / 100)),
                int(frame_width * (bottom_right[0] / 100)), int(frame_height * (bottom_right[1] / 100))
            ], dtype=np.float32)

            if obj_idx < len(last_used_coords):
                points_info = last_used_coords[obj_idx]
                x_percent, y_percent, label_value = points_info
                points = np.array([[int(frame_width * (x_percent / 100)), int(frame_height * (y_percent / 100))]], dtype=np.float32)
                labels = np.array([label_value], dtype=np.int32)
            else:
                points = None
                labels = None

            predictor.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=image_index - 1,
                obj_id=obj_id,
                points=points,
                labels=labels,
                box=box
            )
    else:
        raise ValueError(f"Failed to load the frame at {selected_frame_path}.")

    # 順方向のマスク伝播（指定フレームから最後のフレームまで）
    video_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

        # 伝播が最後のフレームに到達したらループを抜ける
        if out_frame_idx >= len(frame_names) - 1:
            break

    # 逆方向の伝播を実現するために、フレームを逆順にする
    reversed_frame_indices = list(range(0, image_index))[::-1]  # [image_index-1, image_index-2, ..., 0]

    # 逆順のフレーム名リストを作成
    reversed_frame_names = [frame_names[idx] for idx in reversed_frame_indices]

    # 新しいフォルダを作成して逆順フレームを保存
    reversed_video_dir = "./videos_reversed"
    if not os.path.exists(reversed_video_dir):
        os.makedirs(reversed_video_dir)

    for i, frame_name in enumerate(reversed_frame_names):
        reversed_frame_path = os.path.join(video_dir, frame_name)
        new_frame_path = os.path.join(reversed_video_dir, f"{i:05d}.jpg")
        os.system(f"cp {reversed_frame_path} {new_frame_path}")

    # 逆順フレームのディレクトリでセグメンテーション伝播を実行
    reversed_inference_state = predictor.init_state(video_path=reversed_video_dir)
    predictor.reset_state(reversed_inference_state)

    # 再度、指定されたフレームから設定を追加
    for obj_idx, (top_left, bottom_right) in enumerate(stored_boxes):
        obj_id = obj_idx + 1

        # ボックスの再計算
        box = np.array([
            int(frame_width * (top_left[0] / 100)), int(frame_height * (top_left[1] / 100)),
            int(frame_width * (bottom_right[0] / 100)), int(frame_height * (bottom_right[1] / 100))
        ], dtype=np.float32)

        # ポイントとラベルの再設定
        if obj_idx < len(last_used_coords):
            points_info = last_used_coords[obj_idx]
            x_percent, y_percent, label_value = points_info
            points = np.array([[int(frame_width * (x_percent / 100)), int(frame_height * (y_percent / 100))]], dtype=np.float32)
            labels = np.array([label_value], dtype=np.int32)
        else:
            points = None
            labels = None

        # デバッグ出力
        print(f"Setting obj_id {obj_id} with box {box}, points {points}, and labels {labels} for reversed propagation.")

        predictor.add_new_points_or_box(
            inference_state=reversed_inference_state,
            frame_idx=0,  # 逆順の最初のフレームはフレームインデックス0
            obj_id=obj_id,
            points=points,
            labels=labels,
            box=box
        )

    # 逆順のフレーム伝播
    reversed_video_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(reversed_inference_state):
        reversed_video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

        # 伝播が逆方向の最後のフレーム（実際の最初のフレーム）に到達したらループを抜ける
        if out_frame_idx >= len(reversed_frame_names) - 1:
            break

    # 結果を元の順序に戻す
    for rev_idx, frame_idx in enumerate(reversed_frame_indices):
        video_segments[frame_idx] = reversed_video_segments[rev_idx]

    # マスク保存用のフォルダを作成
    mask_folder = '/content/segmentation_results'
    if not os.path.exists(mask_folder):
        os.makedirs(mask_folder)

    # マスクカラー画像保存用フォルダを作成
    mask_color_folder = create_mask_color_folder()

    # グレースケールマスク保存用のフォルダを作成
    grayscale_folder = '/content/grayscale_masks'
    if not os.path.exists(grayscale_folder):
        os.makedirs(grayscale_folder)

    segmented_image_paths = []

    # SVGファイル保存用フォルダを作成
    svg_folder = '/content/mask_svgs'
    if not os.path.exists(svg_folder):
        os.makedirs(svg_folder)

    # 各フレームの結果を表示し、マスクを保存
    plt.close("all")
    for frame_idx, frame_name in enumerate(frame_names):
        image_np = np.array(Image.open(os.path.join(video_dir, frame_name)))

        if image_np is None:
            print(f"Failed to load image at frame {frame_name}. Skipping...")
            continue

        masks_to_apply = video_segments.get(frame_idx, {}).values()
        output_image = apply_all_masks3(image_np, list(masks_to_apply), frame_idx, svg_folder)  # svg_folderを追加

        filename = f"segmented_image_{starting_index + frame_idx:04}.jpg"
        filepath = os.path.join(mask_folder, filename)
        save_masked_image_to_disk(output_image, filepath)

        mask_color_filename = f"mask{starting_index + frame_idx:04}.png"
        mask_color_filepath = os.path.join(mask_color_folder, mask_color_filename)
        save_mask_color_image_to_disk(image_np, masks_to_apply, mask_color_filepath)

        grayscale_filename = f"mask{starting_index + frame_idx:04}.png"
        grayscale_filepath = os.path.join(grayscale_folder, grayscale_filename)
        save_grayscale_mask(image_np, masks_to_apply, grayscale_filepath)

        segmented_image_paths.append(filepath)

        plt.figure(figsize=(6, 4))
        plt.title(f"Frame {frame_idx}")
        plt.imshow(output_image)
        plt.show()

    print(f"Segmentation results saved to {mask_folder}")

    zip_filepath_masks = create_zip_from_masks()
    zip_filepath_maskcolors = create_zip_from_maskcolors()
    zip_filepath_svgs = create_zip_from_svgs()  # SVGのZIPファイル作成
    zip_filepath_grayscale = create_zip_from_grayscale_masks()  # グレースケールマスクZIP

    return zip_filepath_masks, zip_filepath_maskcolors, zip_filepath_svgs, zip_filepath_grayscale, segmented_image_paths, "Segmentation has been applied to all images."

# Gradioインターフェース
def gradio_interface():
    global object_counter
    object_counter = 0  # 初期状態ではオブジェクト0からスタート

    with gr.Blocks() as demo:
        gr.Markdown("## SAM2 GUI for Img Seq (Seg&Ref)")
        gr.Markdown("### How to Reset")
        gr.Markdown(
            "To reset the interface, return to the Colab notebook and go to "
            "**Runtime > Disconnect and delete runtime**, then re-run all cells by selecting "
            "**Runtime > Run all**."
        )

        with gr.Row():
            with gr.Column():
                color_start_dropdown = gr.Dropdown(
                    choices=[(f"{i+1}: {color_labels[i]}", str(i)) for i in range(len(color_labels))],
                   value="0",  # 内部的には0がデフォルト
                    label="Select Starting Color for Object 1"
                )

                upload_btn = gr.Files(label="Upload Images", file_count="multiple", file_types=["image"])
                upload_status = gr.Textbox(label="Upload Status")
                uploaded_images_gallery = gr.Gallery(label="Uploaded Images", show_label=True)
                slider = gr.Slider(label="Frame Selection", minimum=1, maximum=1, step=1, value=1)
                select_btn = gr.Button("Perform Segmentation on This Image")
                displayed_image = gr.Image(label="Selected Image")
                color_status_output = gr.Textbox(label="Selected Start Color", interactive=False)
                score_output = gr.Textbox(label="Mask Score")
                object_list_display = gr.Textbox(label="Object List")

            with gr.Column():
                x_slider = gr.Slider(label="X Coordinate (%)", minimum=0, maximum=100, value=50, step=1)
                y_slider = gr.Slider(label="Y Coordinate (%)", minimum=0, maximum=100, value=50, step=1)
                # Set Top Left と Set Bottom Right を横並びにする
                with gr.Row():
                    top_left_btn = gr.Button("Set Top Left")
                    bottom_right_btn = gr.Button("Set Bottom Right")
                    undo_btn = gr.Button("Undo Box Selection")  # Undo ボタンを追加
                complete_and_add_btn = gr.Button("Complete Segmentation or Add Next Object")
                output_image = gr.Image(label="Segmentation Result")
                start_tracking_btn = gr.Button("Start Tracking")
                all_segmented_images = gr.Gallery(label="Segmentation Results for All Images")
                notification_output = gr.Textbox(label="Notification", visible=False)
                download_zip_masks_output = gr.File(label="Images with mask overlay(ZIP)")
                download_zip_maskcolors_output = gr.File(label="Black background mask(ZIP)")
                download_zip_svgs_output = gr.File(label="Vector mask (SVG) ZIP")
                download_zip_grayscale_output = gr.File(label="Grayscale masks (ZIP)")

        # 複数画像のアップロードイベント（アップロード画像ギャラリーを追加）
        upload_btn.upload(
            fn=upload_images,
            inputs=upload_btn,
            outputs=[upload_status, slider, displayed_image, uploaded_images_gallery]  # ギャラリーを追加
        )

        # 🔽 ドロップダウンの変更イベント接続
        color_start_dropdown.change(
            fn=update_color_start_from_dropdown,
            inputs=color_start_dropdown,
            outputs=color_status_output
        )

        # スライダーで画像を表示
        slider.change(fn=display_image, inputs=slider, outputs=displayed_image)

        # スライダーが動いたときに画像にラインを描画
        x_slider.change(fn=lambda image, x, y: update_image_with_lines(image, x, y),
                        inputs=[displayed_image, x_slider, y_slider],
                        outputs=output_image)
        y_slider.change(fn=lambda image, x, y: update_image_with_lines(image, x, y),
                        inputs=[displayed_image, x_slider, y_slider],
                        outputs=output_image)

        top_left_btn.click(fn=set_top_left, inputs=[x_slider, y_slider], outputs=score_output)
        # 'displayed_image' も 'inputs' に含めて、set_bottom_right に渡す
        bottom_right_btn.click(
            fn=set_bottom_right,
            inputs=[x_slider, y_slider, displayed_image],  # 画像データも追加
            outputs=[output_image, score_output]  # 画像とメッセージを別々に出力
        )

        # 「この画像でセグメンテーションを行う」ボタンが押されたとき
        select_btn.click(fn=update_result_on_select, inputs=slider, outputs=[displayed_image, output_image])

        # Undoボタンのクリック時の挙動を設定、object_counterに応じて画像をリセット
        undo_btn.click(
            undo_box_selection,
            inputs=[displayed_image, output_image, x_slider, y_slider],
            outputs=[output_image, score_output, x_slider]  # x_sliderも更新
        )

        # セグメンテーションを完了し、次のオブジェクトを追加
        complete_and_add_btn.click(fn=complete_and_add_object, inputs=[displayed_image, x_slider, y_slider], outputs=[output_image, score_output, object_list_display])

        # # トラッキングを開始ボタンが押された時、全ての画像にセグメンテーションを適用
        # start_tracking_btn.click(fn=lambda: process_video_with_propagation("/content/output_video.mp4", last_used_coords, stored_boxes, image_index), inputs=[], outputs=[download_zip_masks_output, download_zip_maskcolors_output, all_segmented_images])
        start_tracking_btn.click(
            fn=lambda image_index: process_video_with_propagation("/content/output_video.mp4", last_used_coords, stored_boxes, image_index),
            inputs=[slider],  # スライダーからの入力を受け取る
            outputs=[download_zip_masks_output, download_zip_maskcolors_output, download_zip_svgs_output, download_zip_grayscale_output, all_segmented_images, notification_output]
        )

        notification_output.change(
            fn=None,
            inputs=[],
            outputs=[],
            js="""
            () => {
                // ✅ 非ブロッキング通知（HTML要素として表示）
                const msg = document.createElement('div');
                msg.innerText = "✅ Segmentation completed and results displayed.";
                msg.style.position = "fixed";
                msg.style.bottom = "20px";
                msg.style.right = "20px";
                msg.style.background = "#4CAF50";
                msg.style.color = "white";
                msg.style.padding = "10px 20px";
                msg.style.borderRadius = "10px";
                msg.style.zIndex = 9999;
                msg.style.boxShadow = "0 0 10px rgba(0,0,0,0.3)";
                document.body.appendChild(msg);
                setTimeout(() => msg.remove(), 4000);  // 4秒で自動消去

                // 🎵 チャイム音（即再生）
                const context = new AudioContext();
                const o = context.createOscillator();
                const g = context.createGain();
                o.type = 'sine';
                o.frequency.setValueAtTime(880, context.currentTime);
                o.connect(g);
                g.connect(context.destination);
                o.start();
                o.stop(context.currentTime + 0.2);

                setTimeout(() => {
                    const o2 = context.createOscillator();
                    const g2 = context.createGain();
                    o2.type = 'sine';
                    o2.frequency.setValueAtTime(660, context.currentTime);
                    o2.connect(g2);
                    g2.connect(context.destination);
                    o2.start();
                    o2.stop(context.currentTime + 0.2);
                }, 300);
            }
            """
        )

    return demo

from IPython.display import Audio, Javascript, display
import numpy as np

# チャイム関数
def chime_sound():
    sample_rate = 44100  # サンプルレート
    duration = 0.2  # 各音の長さ（秒）

    t1 = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave1 = 0.5 * np.sin(2 * np.pi * 880 * t1)  # 高音A5

    t2 = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave2 = 0.5 * np.sin(2 * np.pi * 660 * t2)  # 中音E5

    silence = np.zeros(int(sample_rate * 0.05))
    wave = np.concatenate([wave1, silence, wave2])

    return wave

display(Audio(chime_sound(), rate=44100, autoplay=True))

# 🎵 先に音を鳴らす
display(Audio(chime_sound(), rate=44100, autoplay=True))

# 🔔 音の再生後にポップアップを表示（少し遅延）
display(Javascript("""
    setTimeout(() => {
        alert("✅ Your Gradio app is ready! Please check the URL.");
    }, 300);  // 300ms delay to allow audio playback
"""))

# Gradioアプリの起動
gradio_interface().launch(share=True, debug=True)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Tue Jun 10 02:48:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0       

<IPython.core.display.Javascript object>

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e8acd6a610739a2273.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Format of uploaded image 1: JPEG
Image is already in JPEG format.: /tmp/gradio/e7c2459a5346951850a7684b99387ddf4798a72cf2ccbc72c9e45e30915939d3/image0001.jpg
Format of uploaded image 2: JPEG
Image is already in JPEG format.: /tmp/gradio/dea535a4dd86ab9f004128a5c21627466359c963294c89a93e2c189b69754e0c/image0002.jpg
Format of uploaded image 3: JPEG
Image is already in JPEG format.: /tmp/gradio/8e866a85fe7f19c123cb60d45abb94ea6615b5b1ba389615e8b04d90291222fb/image0003.jpg
Format of uploaded image 4: JPEG
Image is already in JPEG format.: /tmp/gradio/00ad38d425bec07e0aa96685c0d287e358501800c305384da78536d93db45413/image0004.jpg
Format of uploaded image 5: JPEG
Image is already in JPEG format.: /tmp/gradio/e6ce6f8e83ade6a85e110c997907fb535d46f3799caf318431e43add5fdc7c1f/image0005.jpg
The mask was applied.: 0 masks.
The mask was applied.: 0 masks.
The mask was applied.: 0 masks.
NumPy array converted to JPEG and saved as: /tmp/tmp7zq8rrnc.jpg
Image saved to: /tmp/tmp9j03ea72/0.jpg
Bottom-rig

frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 19.73it/s]


...

frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 23.00it/s]


Setting obj_id 1 with box [546. 215. 829. 318.], points None, and labels None for reversed propagation.
Setting obj_id 2 with box [829. 318. 939. 369.], points None, and labels None for reversed propagation.


propagate in video:   0%|          | 0/1 [00:00<?, ?it/s]

create_mask_color_folder called
Folder created at: /content/maskcolors
Original mask at index 0: [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
Resized mask at index 0: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Mask size: (430, 1092), Image size: (430, 1092, 3)
Color BGR at index 0: (0, 0, 255)
Expanded mask at index 0: (430, 1092, 3)
Mask BGR at index 0: [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
Original mask at index 1: [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [

Original mask at index 0: [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
Resized mask at index 0: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Mask size: (430, 1092), Image size: (430, 1092, 3)
Color BGR at index 0: (0, 0, 255)
Expanded mask at index 0: (430, 1092, 3)
Mask BGR at index 0: [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
Original mask at index 1: [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0

Copyright (c) 2024 Satoru Muro. All rights reserved.
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.